In [1]:
# Multiple outputs per cell in Jupyter 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
%%javascript
// Evitar autoscroll.
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
# import numpy as np
# # from toolbox import *
# import toolbox as tb
# from skeleton import *
# import pandas as pd
# from gensim.models import FastText
# %load_ext autoreload
# %autoreload 2
# import os
# import tqdm
# from microtc.utils import tweet_iterator
# import datetime
# from collections import Counter
# import gc
# from collections import defaultdict

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from integrado import *

In [13]:
tokens=Tokens().tokenize("data/geo-mx-2004_min.json",replace=True,showProgressEach=10000)

** Processing data/geo-mx-2004_min.json ...
2020-05-12 19:16:27.990884 :: 0.000193 - Processing item #0 
** Processed 10000 lines. Saved to /home/jovyan/public/B6_ProyectoIntegrador/data/geo-mx-2004_min.npy.


In [25]:
index_=Index().computePostingLists(tokens,showProgressEach=10000)
# index_.postlists

** Counting TF & DF ...
2020-05-12 19:18:55.281691 :: 0.001182 - Processing item #0 
** Calculating TF-IDF...
2020-05-12 19:18:55.935591 :: 0.655082 - Processing item #0 
** Sorting posting lists...
2020-05-12 19:18:56.539612 :: 1.259103 - Processing item #0 
2020-05-12 19:18:56.634560 :: 1.354051 - Processing item #10000 


In [26]:
len(index_.postlists.keys())

10383

In [33]:
ds=index_.o('hoy no me puedo levantar',isText=True)
print(len(ds),ds[:5])
docs=tokens.getDocs(ds)
print(len(docs),docs[:5])

619 [3, 5, 9, 45, 54]
619 [['obvi', 'hoy', 'cumpl', 'anos', 'apag', 'voz'], ['hoy', 'cumpl', 'ano', 'plan', 'des', 'secret', 'gent', 'sabi', 'haci', 'sent', 'bien', 'aunqu', 've', 'tan', 'cercan'], ['amig', 'seguidor', 'tod', 'person', 'respet', 'jueg', 'decision', 'jug', 'pued', 'influ', 'ser', 'president', 'estudiantil', 'hag', 'ayud'], ['pregunt', 'quer', 'hoy'], ['pregunt', 'investig', 'aprend', 'pued']]


In [ ]:
gc.collect()
import sys
local_vars = list(locals().items())
tot=0
vars_=[]
for var, obj in local_vars:
    mem=sys.getsizeof(obj)
    tot+=mem
    vars_.append((mem,var))
print("Total =",tot)
vars_.sort(reverse=True)
print(vars_)